In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> 딥러닝 성능 올리기 
-학습을 더 많이 
-layer을 더 쌓기
-이미지 손실 줄이기,이미지 사이즈 적당히 줄여주기,이미지 사이즈를 늘려주는게 점수 개선이된다. 

zip파일이고, csv 아님 

In [ ]:
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip

In [ ]:
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

그림 하나 하나 열어보기 

In [ ]:
from PIL import Image

In [ ]:
Image.open('train/cat.1.jpg')

사진이 있고 모델을 통해 학습한다~ 
머신러닝할때 1. 데이터전처리 (데이터 프레임을 만들어서 칼럼 2개 -> 만드고임/ 하나는 주소 하나는 정답값) 2.

In [ ]:
#데이터 접근##
import glob 
glob.glob('train/*') #.jpg는 안적어도 됨


In [ ]:
train=pd.DataFrame({'path':glob.glob('train/*')})

In [ ]:
train

* apply(): 딥러닝 머신러닝에 자주 사용. 괄호 안에서 함수 만들기 

In [ ]:
train['path'].apply(lambda x:x.split('/'))

In [ ]:
train['path'].apply(lambda x:x.split('/')[1])

In [ ]:
train['path'].apply(lambda x:x.split('/')[1].split('.')[0])

In [ ]:
train['target']=train['path'].apply(lambda x:x.split('/')[1].split('.')[0])

In [ ]:
train
train.head()

학습을 할 수 있도록 전처리를 간단하게,라이브러리 가져와서 작업 

In [ ]:
test=pd.DataFrame({'path':glob.glob('test/*')})
test

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
IDG=ImageDataGenerator(horizontal_flip=True) 
#이미지 칼럼을 맞춰준다. 우리 데이터의 차원이 항상 똑같아야 한다. 이미지에서는 사이즈가 다르기 때문에 통일해주고자 한다.
#이미지 용량 때문에 batch 조절 해주면서 진행해야 한다. 다른 변환 없이 자동으로 내부적으로 이미지 처리 해주고 얘를 통과해서 트레인 셋에서 학습할거임
#augmentation 을 통해 데이터 증식 시키기,좌우반전 이런거 체크 -> 줌인 줌앗,노이즈, 위아래반전 등이 있다. 
#트레인과 테스트 패턴을 체크하고 나서 추가쓰,augumentation넣을거면 epoch도 늘려주기 -> 원본 데이터 모두 학습시키기 위해서 

In [ ]:
train_generator=IDG.flow_from_dataframe(train, x_col='path',y_col='target',target_size=(300,300),batch_size=32) #dataset 이름,x_column, y_column, batch_size=100장씩 
test_generator=IDG.flow_from_dataframe(test,x_col='path',y_col=None,target_size=(300,300),batch_size=32,class_mode=None,shuffle=False)#class_mode=none 기본값이 categorical,testset에는 정답값이 없으니까  test셋에는 꼭 필요함,shuffle은 안하면 제출시 원하는 데이터의 아이디에 맞게 정답값이 섞이게 됨 

모델링 / 이미지 학습 중 

1.Inception v3

In [ ]:
#from keras import Sequential #층을 쌓는다는 느낌으로 
#from keras.applications.inception_v3 import InceptionV3 #최대한 최신 모델 가져오기
#from keras.layers import *

In [ ]:
#model=Sequential()
#model.add(InceptionV3(weights='imagenet',include_top=False,pooling='avg')) 

#model.add(Dense(2,activation='softmax'))

#model.compile(optimizer='adam', metrics=['acc'], loss='categorical_crossentropy')



2. 일반 이미지 모델 

In [ ]:
#모델 층 쌓기 
#model =Sequential() 

#model.add(Conv2D(32,(3,3),activation='relu',input_shape=(100,100,3))) 
#model.add(Flatten())
#model.add(Dense(2,activation='softmax')) 
#model.summary()
#model.compile(optimizer='adam', metrics=['acc'], loss='categorical_crossentropy') 

#모델 층 쌓기 
model =Sequential() #모델 선언 시작할게, 이제 입력층이 나올거다 

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(100,100,3))) #입력층으로 들어갈 때 cnn층이 필요하다.convolution layer인데 2차원, 보통은 2의 제곱이 들어간다. 얼마나 많은 특징을 추출하는지가 효율적인거
#알아서 32개의 특징을 추출한다. #(3,3)이미지 처리에 있어서 고정,가장 효율적임 #비선형성을 학습할 수 있어야한다. #주로 relu를 많이 쓴다. #input_size제시해주고 채널까지-> 안맞춰주면 오류가 생긴다. 
#Flatten : 매개변수 층이 필요함, 층이 안맞아서 연결해주는 역할 
model.add(Flatten())#Denselayer1차원이니까 펴주는 역할 
model.add(Dense(2,activation='softmax')) 
#최소한 층이 2개가 있어야 한다. 입력층-출력층 출력층은 거의 항상 Denselayer(), 
#맨 마지막 출력층인 경우에만 softmax. 밖으로 나갈 때 확률값으로 출력해야함
#새로운 데이터가 들어왔을 때는 어떤 class인지 알 수 있어야 한다.정답 클래스의 개수(고양이의 확률 or 개의 확률 )가 필요하다. 이건 이진 분류니까 2 
#출력물 지날 때 활성화 함수-> 확률로 바뀌어야함
#sigmoid/ softmax 모두 가능/ 멀티 클래스일때 softmax가 편하다. 출력층이 softmax일때 모델링이 잘 됨!
model.summary()
model.compile(optimizer='adam', metrics=['acc'], loss='categorical_crossentropy') # 모델을 컴파일 시켜준다. 학습을 해나가면서 최적화하기위함/ 손실=>cross이거 고정 특히 다중 multiclass에서 고정임, 행렬, 최적화 metrics->평가 방식,정확도의 직관성에 관해서 
#y=wx+b(y=정답값,x=input값, 정답을 잘 맞출 수 있게 하는 W를 찾아나가는게 핵심,처음에는 아무값으로 설정되다가 학습을 통해서 업데이트 된다.w=로스 및 손실함수를 통해 가중치를 점점 조절해나간다.),optimizer=gradient_descent

#가중치=random으로 시작했는데 Imagenet에서 들고옴.이미 여기에서 모델을 돌린상태의 가중치를 들고 와서 우리 모델에 사용하겠다. 
#고양이나 강아지 자체가 imagenet에서 이미 클래스들로 들어와있음 
#기본적으로 사진's 공통점, 픽셀값으로 들어가있다, 직선정보나 곡선,명암등에 있어서 기본적으로 존재함. 
#배경정보가 클래스 정보에 영향을 미치는데 이미 물체 이런거에 관한 가중치계산이 끝났음, 배경정보를 무시할 수 있다. 이를 전이학습이라 하는거여
#include_top:딥러닝 구조 상으로 입력층이 아래인데, 이는 출력층을 포함하지 않겠다는 뜻. 출력층을 직접 만들어주기 위해서 안 포함. 
#Flatten->1차원으로 펼쳐주는 역할, 정보가 소실되기도 함, 그래서 pooling avg통해 지엽적인 정보를 덜 날리게

model.add(Dense(2,activation='softmax'))
#고양이 강아지만 나눌려고
model.compile(optimizer='adam', metrics=['acc'], loss='categorical_crossentropy')
#출력층 제외하고 InceptionV3로 바로 진행하겠다. 요 안에 conv maxpooling 다 들어가있음~



Convolution layer : Conv2D(필터로 특징 추출해주는것, CNN에서는 특징을 뽑아 이미지 처리하기 때문에 )
tf.keras.layers.Conv2D(
    filters,
    kernel_size,
    strides=(1, 1),
    padding="valid",
    data_format=None,
    dilation_rate=(1, 1),
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
>>> input_shape = (4, 28, 28, 3)
>>> x = tf.random.normal(input_shape)
>>> y = tf.keras.layers.Conv2D(
... 2, 3, activation='relu', padding="same", input_shape=input_shape[1:])(x)
>>> print(y.shape)
(4, 28, 28, 2)

efficinet B1 

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from efficientnet.tfkeras import EfficientNetB1
from keras import Sequential
from keras.layers import * 
from keras.optimizers import SGD
model=Sequential()
model.add(EfficientNetB1(weights='imagenet',pooling='avg',include_top=False))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=0.01,nesterov=True,momentum=0.9),metrics=['acc'])


earlystopping -> overfitting이 또 생길 수 있음
modelcheckpoint: 다시 예측하기 전에
ReduceLR-> learning rate 조절, learning rate가 너무 크면, global_min(x=0인지점)으로 못가고 주변을 맴돌게 됨 learning rate가 너무커서 최적이 안되는거임
이번엔 optimizer바꾸기 adam도 좋은데, SGD 파라미터 조절 필수임 1. learning rate/2.nestrov,방향 자체를 고려해서 다음 방향 설정해야하는데 너무 급변하는데 학습하변 값이 튈 수 있으니까 예전의 방향 고려해서 벡터 설정 /3.momentum 예전 관성을 얼마나 유지할거냐

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
es=EarlyStopping(patience=5)
mc=ModelCheckpoint('best.h5',save_best_only=True)
RL=ReduceLROnPlateau(patience=3,factor=0.2) 

RL이 더 커버리면,early stopping되기도 전에 멈춰버리니까, factor은 몇 정도 줄어들꺼냐 

In [ ]:
#모델 학습하기 
model.fit(train_generator,callbacks=[es,mc,RL],epochs=3) #이미 클래스가 분류되어있어서 이미 y값이 정답값도 적용되어있다! 

batch=100개씩 들어옴,25000개 중에 100개, 그리고 점점 로스 줄여주고 w 바꿔주는거여

In [ ]:
#모델예측하기
result=model.predict(test_generator,verbose=1) #괄호에 test할 것들을 넣으면 된다. 

In [ ]:
sub=pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')

In [ ]:
sub

In [ ]:
train_generator.class_indices

In [ ]:
result

In [ ]:
sub['label']=result[:,1].clip(0.005,0.995) #:모든데이터 ,1은 몇번째 열, 0.005보다 작은애들은 0.005로 바꾸고 0.995 이상으로 큰 애들은 이 값으로 바꾸어버리겠다

In [ ]:
sub

제출시 id에 맞게 되어야하는데  match가 안맞음

In [ ]:
test

In [ ]:
sub['id']=test['path'].apply(lambda x:x.split('/')[1].split('.jpg')[0])

id에 맞는 label을 맞춰주는게 엄청 중요하다. 

In [ ]:
sub

In [ ]:
sub.to_csv('sub_catanddog.csv',index=0)

-앙상블 모델: 점수 잘나온 모델에 점수를 준다
-모델 2개를 branch로 만들어서 컴퓨팅 진행,functional api 